# Data Import, Cleaning, and EDA for AirBnB listings in Mexico City
from insideairbnb.com

## Import Libraries

In [128]:
import pandas as pd
import numpy as np
import seaborn as sns

## Import Data
Free Data is available quarterly so I have imported listings one year back starting in June 2022

In [129]:
june = pd.read_csv("./data/listings_june.csv")
june["last_scraped"].unique()

array(['2022-06-21', '2022-06-22'], dtype=object)

In [130]:
march = pd.read_csv("./data/listings_march.csv")
march["last_scraped"].unique()

array(['2022-03-27', '2022-03-26'], dtype=object)

In [131]:
dec = pd.read_csv("./data/listings_dec.csv")
dec["last_scraped"].unique()

array(['2021-12-26', '2021-12-25', '2021-12-29'], dtype=object)

In [132]:
sep = pd.read_csv("./data/listings_sept.csv")
sep["last_scraped"].unique()

array(['2021-09-30', '2021-09-29', '2021-09-28', '2021-10-06'],
      dtype=object)

## Drop features with high proportion of missing values

In [133]:
# Determine NaN rates and drop features with > 80% NaNs
thresh = 0.95
nan_rates = june.isna().sum(axis=0).sort_values(ascending=False) / float(len(june))
june.drop(nan_rates[nan_rates > thresh].index, axis=1, inplace=True)
print(
    f"Removed {len(nan_rates[nan_rates > thresh])} features with > {thresh*100}% missing values"
)

# Also drop rows with all NaNs
june.dropna(how="all", inplace=True)

Removed 4 features with > 95.0% missing values


In [134]:
june.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 

In [135]:
june['host_response_time'].unique()

array([nan, 'a few days or more', 'within an hour', 'within a few hours',
       'within a day'], dtype=object)

In [136]:
# Must revisit this drop since there is useful info here that I'm dropping for simplicity
june.drop(columns=['property_type', 'amenities', 'calendar_last_scraped'], inplace=True)

In [137]:
june = june.astype(
    {
        "last_scraped": "datetime64",
        "host_since": "datetime64",
        'first_review': 'datetime64',
        'last_review': 'datetime64',
    }
)

In [138]:
june["host_response_rate"] = (
    june["host_response_rate"].str.replace("%", "").fillna(0).astype("float") / 100.0
)

In [139]:
june["host_acceptance_rate"] = (
    june["host_acceptance_rate"].str.replace("%", "").fillna(0).astype("float") / 100.0
)

In [140]:
june["host_is_superhost"] = (
    june["host_is_superhost"].map({"f": 0, "t": 1}).astype("bool")
)

In [141]:
june["host_identity_verified"] = (
    june["host_identity_verified"].map({"f": 0, "t": 1}).astype("bool")
)

In [142]:
june['bathrooms_text'] = june["bathrooms_text"].map(
    {
        "3 baths": "3 a",
        "1 bath": "1 a",
        "1 private bath": "1 a",
        "5.5 baths": "5.5 a",
        "1 shared bath": "1 b",
        "1.5 baths": "1.5 a",
        "1.5 shared baths": "1.5 b",
        "2 baths": "2 a",
        "5 baths": "5 a",
        "4.5 baths": "4.5 a",
        "2.5 baths": "2.5 a",
        "3.5 baths": "3.5 a",
        "2 shared baths": "2 b",
        "5.5 shared baths": "5.5 b",
        "6 baths": "6 b",
        "3 shared baths": "3 b",
        "8 shared baths": "8 b",
        "2.5 shared baths": "2.5 b",
        "Half-bath": "0.5 a",
        "3.5 shared baths": "3.5 b",
        "4 baths": "4 a",
        "10 baths": "10 a",
        "5 shared baths": "5 b",
        "16 baths": "16 a",
        "4 shared baths": "4 a",
        "0 baths": "0 a",
        "7 baths": "7 a",
        "20 baths": "20 a",
        "4.5 shared baths": "4.5 b",
        "0 shared baths": "0 b",
        "50 baths": "50 a",
        "10 shared baths": "10 b",
        "Shared half-bath": "0.5 b",
        "6.5 shared baths": "6.5 b",
        "8.5 baths": "8.5 a",
        "7.5 baths": "7.5 a",
        "9 baths": "9 a",
        "11.5 baths": "11.5 a",
        "8 baths": "8 a",
        "Private half-bath": "0.5 a",
        "14 baths": "14 a",
        "11 baths": "11 a",
        "6.5 baths": "6.5 a",
        "6 shared baths": "6 b",
        "8.5 shared baths": "8.5 b",
        "13 baths": "13 a",
        "15 shared baths": "15 b",
        "17 baths": "17 a",
        "12 shared baths": "12 b",
        "7.5 shared baths": "7.5 b",
        "7 shared baths": "7 b",
        "9.5 baths": "9.5 a",
        "30 baths": "30 a",
        "26 baths": "26 a",
    }
)

In [143]:
june[['bathrooms_count', 'bathrooms_type']] = june['bathrooms_text'].str.split(expand=True)
june['bathrooms_count'] = june['bathrooms_count'].astype('float')
june['bathrooms_type'] = june['bathrooms_type'].astype('category')

In [144]:
june['price'] = june['price'].str.replace('[\$,]', '', regex=True).astype('float')

In [145]:
june["instant_bookable"] = (
    june["instant_bookable"].map({"f": 0, "t": 1}).astype("bool")
)

In [146]:
june["has_availability"] = (
    june["has_availability"].map({"f": 0, "t": 1}).astype("bool")
)

In [147]:
june.groupby('neighbourhood').mean()['price'].sort_values(ascending=False).head(10)

neighbourhood
CDMX , Ciudad de Mexico, Mexico                   20000.000000
Colonia Condesa, Mexico City, Mexico              10933.000000
Lomas de Chapultepec, Ciudad de México, Mexico    10121.000000
San Miguel Ajusco, Ciudad de México, Mexico        9870.333333
Cuernavaca, Morelos, Mexico                        9127.000000
El Capulín, Morelos, Mexico                        8500.000000
CDMX, D.F., Mexico                                 6084.000000
Mexico, CDMX, Mexico                               4442.000000
Colonia Hipódromo, Ciudad de México, Mexico        4375.000000
San Agustin Etla, Oaxaca, Oax., Oaxaca, Mexico     4056.000000
Name: price, dtype: float64

In [148]:
june.corr()['price'].sort_values().head()

longitude                                      -0.020074
calculated_host_listings_count_private_rooms   -0.014550
number_of_reviews                              -0.013949
number_of_reviews_ltm                          -0.012835
number_of_reviews_l30d                         -0.012649
Name: price, dtype: float64

In [149]:
june.corr()['price'].sort_values(ascending=False).head(6)

price                                          1.000000
accommodates                                   0.103180
bathrooms_count                                0.091676
bedrooms                                       0.080340
beds                                           0.076775
calculated_host_listings_count_entire_homes    0.035844
Name: price, dtype: float64

In [150]:
# Keep features related to property only
june.drop(
    columns=[
        "id",
        "listing_url",
        "scrape_id",
        "last_scraped",
        "name",
        "description",
        "neighborhood_overview",
        "picture_url",
        "host_id",
        "host_url",
        "host_name",
        "host_since",
        "host_location",
        "host_about",
        "host_response_time",
        "host_response_rate",
        "host_acceptance_rate",
        "host_is_superhost",
        "host_thumbnail_url",
        "host_picture_url",
        "host_neighbourhood",
        "host_listings_count",
        "host_total_listings_count",
        "host_verifications",
        "host_has_profile_pic",
        "host_identity_verified",
        "minimum_nights",
        "maximum_nights",
        "minimum_minimum_nights",
        "maximum_minimum_nights",
        "minimum_maximum_nights",
        "maximum_maximum_nights",
        "minimum_nights_avg_ntm",
        "maximum_nights_avg_ntm",
        "has_availability",
        "availability_30",
        "availability_60",
        "availability_90",
        "availability_365",
        "number_of_reviews",
        "number_of_reviews_ltm",
        "number_of_reviews_l30d",
        "first_review",
        "last_review",
        "review_scores_rating",
        "review_scores_accuracy",
        "review_scores_cleanliness",
        "review_scores_checkin",
        "review_scores_communication",
        "review_scores_location",
        "review_scores_value",
        "instant_bookable",
        "calculated_host_listings_count",
        "calculated_host_listings_count_entire_homes",
        "calculated_host_listings_count_private_rooms",
        "calculated_host_listings_count_shared_rooms",
        "reviews_per_month",
        "bathrooms_text",
        "neighbourhood",
        "neighbourhood_cleansed",
        "bathrooms_type",
    ],
    inplace=True,
)

In [151]:
june.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21669 entries, 0 to 21668
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   latitude         21669 non-null  float64
 1   longitude        21669 non-null  float64
 2   room_type        21669 non-null  object 
 3   accommodates     21669 non-null  int64  
 4   bedrooms         21000 non-null  float64
 5   beds             21272 non-null  float64
 6   price            21669 non-null  float64
 7   bathrooms_count  21617 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.3+ MB


In [152]:
june = pd.get_dummies(june, columns=['room_type'])

In [153]:
from sklearn.decomposition import PCA

In [154]:
X = june.select_dtypes(['number']).drop(columns=['price']).fillna(0)

In [155]:
pca = PCA()
pca.fit(X)

PCA()

In [156]:
pd.DataFrame(data=pca.explained_variance_ratio_, index=X.columns).sort_values(by=0, ascending=False).head(20)

,0
latitude,7.367207e-01
longitude,1.390870e-01
accommodates,5.550552e-02
bedrooms,3.576844e-02
beds,2.948663e-02
bathrooms_count,2.193745e-03
room_type_Entire home/apt,9.163933e-04
room_type_Hotel room,1.907161e-04
room_type_Private room,1.308098e-04
room_type_Shared room,6.001090e-34
